In [2]:
import os
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import DirectoryLoader
# from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from yandex_chain import YandexEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_community.document_loaders.csv_loader import CSVLoader

In [32]:
csv_file_name = 'csv_files/history.csv'
full_path = os.path.join("..", "data", csv_file_name)

In [47]:
import pandas as pd
import numpy as np
df = pd.read_csv(full_path, index_col=0)
df.head()


,Код,Категория,Время регистрации,Рабочая группа,Краткое описание,Описание,Решение,Аналитика 1,Аналитика 2,Аналитика 3
0,C03919673,Изменение,16.08.2021 2:16:11,изм 1с erp 2.0,Без СППР. 2021 ЯФИ Заполнение ТЧ по реквизитам,в УПП была кнопка для заполнения колонки в ТЧ ...,Обращение от XXXX года неактуально.,Менеджер услуги,Другой вопрос,NaN
1,C04273748,Изменение,03.12.2021 1:09:46,изм 1с erp 2.0 запросы на развитие,Без СППР. FW: 2.0 _АО_ нет графы количества,В авансовых отчетах программы XС X.X нет графы...,добрый день! Доработка функционала XC ERP X.X...,"Казначейство, взаиморасчеты",Расчеты с персоналом,Авансовые отчеты
2,C04274763,Изменение,03.12.2021 3:52:24,изм 1с erp 2.0 запросы на развитие,Без СППР. 2023 ЗИ - 1.2 ФУНКЦИОНАЛ 1С ЕРП2. ...,Система «срезает» часть номера ЗРС при отражен...,Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Заявки на оплату
3,C04331762,Изменение,20.12.2021 1:46:04,изм 1с erp 2.0 запросы на развитие,Без СППР. Настройки для отчета по дебиторско...,Добрый день! Отправляю обращение проектной ко...,Добрый день. а. В системе созданы отчеты по де...,"Казначейство, взаиморасчеты",Взаиморасчеты,Отчеты
4,C04393042,Изменение,14.01.2022 12:45:38,изм 1с erp 2.0 запросы на развитие,Без СППР. 1.2 ФУНКЦИОНАЛ 1С ЕРП2: в дополн...,"Добрый день, Признак ``Списать на расходы`` ...",Доработка функционала возможна в рамках запрос...,"Казначейство, взаиморасчеты",Казначейство,Платежные документы


In [70]:
#load documents from Pandas dataframe for insertion into database
from langchain.document_loaders import DataFrameLoader

# page_content_column is the column name in the dataframe to create embeddings for
loader = DataFrameLoader(df, page_content_column = 'Описание')
docs = loader.load()


In [71]:
docs[0]

Document(page_content='в УПП была кнопка для заполнения колонки в ТЧ документа одним значением На скриншотах документы ``Передача материалов в кладовую``, ``Заказ материалов в производство`` возможна реализация в рамках официального ЗИ - нужно оценить трудозатраты --------------------- Добрый день! Имеется ввиду в документах по строкам «заполнение по реквизитам» , как было в УПП, например в заказе материалов не выбирать   в каждой позиции дату отгрузки.  X. В документах, например, «заказ поставщику», нет возможности как в XСERP заполнять строки аналитик «по реквизитам», просим добавить такую возможность в документы. Просьба пояснить, что имеется ввиду  ----- ', metadata={'Код': 'C03919673', 'Категория': 'Изменение', 'Время регистрации': '16.08.2021 2:16:11', 'Рабочая группа': 'изм 1с erp 2.0', 'Краткое описание': 'Без СППР. 2021 ЯФИ Заполнение ТЧ по реквизитам', 'Решение': 'Обращение от XXXX года неактуально.', 'Аналитика 1': 'Менеджер услуги', 'Аналитика 2': 'Другой вопрос', 'Аналитик

In [72]:
docs[0].metadata['Аналитика 3']

nan

In [74]:
'nan' == str(docs[0].metadata['Аналитика 3'])

True

In [53]:
text_splitter=RecursiveCharacterTextSplitter( # TODO: semantic chunking
    chunk_size=1000, # TODO: tune this
    chunk_overlap=70
)

chunks = text_splitter.split_documents(docs)

In [54]:
chunks[1]

Document(page_content='В авансовых отчетах программы XС X.X нет графы для проставления количества суточных и количества дней проживания в гостинице', metadata={'Код': 'C04273748', 'Категория': 'Изменение', 'Время регистрации': '03.12.2021 1:09:46', 'Рабочая группа': 'изм 1с erp 2.0 запросы на развитие', 'Краткое описание': 'Без СППР.  FW: 2.0 _АО_ нет графы количества', 'Решение': 'добрый день!  Доработка функционала XC ERP X.X возможна только в рамках запроса на изменение (ЗИ), инициатором обязательно должен быть предоставлен расчет экономического эффекта от реализации ЗИ. Если требования по доработке авансового отчета актуальны, то для реализации ЗИ необходимо направить новое обращение с вложением расчета экономического эффекта от реализации ЗИ. Во вложении направляем памятку по оценке экономической эффективности деятельности.', 'Аналитика 1': 'Казначейство, взаиморасчеты', 'Аналитика 2': 'Расчеты с персоналом', 'Аналитика 3': 'Авансовые отчеты'})

In [55]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")
db = FAISS.from_documents(chunks, embeddings)

/home/yagor/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [56]:
dir_name = 'structured_vdb'
all_path_dir = os.path.join("..", "data", dir_name)
if not os.path.exists(all_path_dir):
    os.makedirs(all_path_dir)

In [57]:
faiss_vdb = "index"

In [58]:
db.save_local(all_path_dir, faiss_vdb)

### Тестирование

In [59]:
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru")

In [60]:
# /home/yagor/Рабочий стол/atomic/AtomHack/data/unstructured_vdb
db2 = FAISS.load_local(
    os.path.join("..", "data", 'structured_vdb'),
    embeddings,
    allow_dangerous_deserialization = True
)

In [78]:
query = "нет графы для п суточных и количества дней проживания"
docs = db2.similarity_search(query)

In [79]:
docs[0]

Document(page_content='В авансовых отчетах программы XС X.X нет графы для проставления количества суточных и количества дней проживания в гостинице', metadata={'Код': 'C04273748', 'Категория': 'Изменение', 'Время регистрации': '03.12.2021 1:09:46', 'Рабочая группа': 'изм 1с erp 2.0 запросы на развитие', 'Краткое описание': 'Без СППР.  FW: 2.0 _АО_ нет графы количества', 'Решение': 'добрый день!  Доработка функционала XC ERP X.X возможна только в рамках запроса на изменение (ЗИ), инициатором обязательно должен быть предоставлен расчет экономического эффекта от реализации ЗИ. Если требования по доработке авансового отчета актуальны, то для реализации ЗИ необходимо направить новое обращение с вложением расчета экономического эффекта от реализации ЗИ. Во вложении направляем памятку по оценке экономической эффективности деятельности.', 'Аналитика 1': 'Казначейство, взаиморасчеты', 'Аналитика 2': 'Расчеты с персоналом', 'Аналитика 3': 'Авансовые отчеты'})

In [81]:
print('Обращение:' , docs[0].page_content)
print('Решение:', docs[0].metadata["Решение"])
category = docs[0].metadata["Аналитика 3"] if 'nan' != str(docs[0].metadata['Аналитика 3']) else docs[0].metadata["Аналитика 2"]
if 'nan' == category:
    category = docs[0].metadata["Аналитика 1"]
print("Категория:", category)

Обращение: В авансовых отчетах программы XС X.X нет графы для проставления количества суточных и количества дней проживания в гостинице
Решение: добрый день!  Доработка функционала XC ERP X.X возможна только в рамках запроса на изменение (ЗИ), инициатором обязательно должен быть предоставлен расчет экономического эффекта от реализации ЗИ. Если требования по доработке авансового отчета актуальны, то для реализации ЗИ необходимо направить новое обращение с вложением расчета экономического эффекта от реализации ЗИ. Во вложении направляем памятку по оценке экономической эффективности деятельности.
Категория: Авансовые отчеты
